# Project Five 
# Game of Thrones - Season Seven

## Importing and Cleaning

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from nltk.tag import pos_tag
import re

Using Theano backend.


In [2]:
pd.set_option('display.max_columns', 135)
pd.set_option('display.max_rows', 135)

In [3]:
df_s7 = pd.read_json('/Users/zoemarkovits/Metis/project_five_got/scrape/season_seven.json')

In [4]:
df_s7[['title','script_text']]

,title,script_text
0,[Dragonstone],"[\n, \nWALDER FREY: You're wonder why I brough..."
1,[Stormborn],"[\n, \nTYRION: On a night like this, you came ..."
2,[The Queen's Justice],"[\n, \nTYRION: The bastard of Winterfell., \n,..."
3,[The Spoils of War],"[\n, \nJAIME: Halt that wagon!, \n, \nJAIME o..."
4,[Eastwatch],"[\n, \nJAIME: You could have killed me., \n, ..."
5,[Beyond the Wall],"[\n, INT. DRAGONSTONE - CHAMBER OF THE PAINTED..."


In [5]:
df_s7['title'] = df_s7['title'].map(lambda x: str(x)[1:-1])

In [6]:
df_s7.title

0            u'Dragonstone'
1              u'Stormborn'
2    u"The Queen's Justice"
3      u'The Spoils of War'
4              u'Eastwatch'
5        u'Beyond the Wall'
Name: title, dtype: object

In [7]:
names7_01 = ['WALDER FREY','ALL','WALDER','ARYA','EDD','MEERA','BRAN','JON','ROBETT','LYANNA','TORMUND','YOHN',
             'SANSA','WOLKAN','JAIME','CERSEI','EURON','MARWYN','SAM','BRIENNE','PETYR','MAN','MAN #2','MAN #3',
             'THOROS','SANDOR','BERIC','GILLY','JORAH','DAENERYS']

names7_02 = ['TYRION','VARYS','DAENERYS','GREY WORM','MELISANDRE','MISSANDEI','SANSA','JON','DAVOS','CERSEI',
             'RANDYLL','QYBURN','JAIME','DICKON','MARWYN','JORAH','SAMWELL','YARA','ELLARIA','OLENNA','ELLIA',
             'MAN #1','MAN #2','HOT PIE','ARYA','WOLKAN','YOHN ROYCE','ROBETT GLOVER','LYANNA MORMONT','BAELISH',
             'NYMERIA','OBARA','TYENE','THEON','EURON']

names7_03 = ['TYRION','JON','DAVOS','MISSANDEI','VARYS','MELISANDRE','DAENERYS','CAPTAIN','THEON','EURON','CERSEI',
             'JAIME','QYBURN','MESSENGER','TYCHO','SANSA','WOLKAN','YOHN','BAELISH','GUARD','BRAN','MARWYN',
             'JORAH','SAMWELL','GREY WORM','OLENNA']

names7_04 = ['JAIME','BRONN','RANDYLL','DICKON','TYCHO','CERSEI','BAELISH','BRAN','MEERA','MAN 1','ARYA','MAN 2',
             'SANSA','PODRICK','BRIENNE','MISSANDEI','DAENERYS','JON','TYRION','DAVOS','THEON']

names7_05 = ['JAIME','BRONN','DAENERYS','RANDYLL','TYRION','DICKON','QYBURN','CERSEI','JON','GUARD','JORAH','BRAN',
             'MAESTER 1','MARWYN','SAMWELL','MAESTER 2','VARYS','DAVOS','ROBETT','YOHN','SANSA','ARYA','GENDRY',
             'GUARD 1','GUARD 2','GILLY','SAM','WOLKAN','BAELISH','TORMUND','BERIC','THOROS','SANDOR']

names7_06 = ['JON','GENDRY','TORMUND','BERIC','THOROS','SANDOR','JORAH','BENJEN','BRIENNE','BAELISH','ARYA',
             'SANSA','DAENERYS','TYRION']

names7_07 = []

### Episode One

In [8]:
#df_s7.script_text[0]

In [9]:
script_text7_01 = df_s7.script_text[0]
print type(script_text7_01)

#script_text7_01

<type 'list'>


In [10]:
script_split = [i.split('\n') for i in script_text7_01]
#script_split

In [11]:
string_script_text = ' '

for i in script_split:
    for e in i:
        string_script_text = string_script_text + ' ' + e

In [12]:
#string_script_text

In [13]:
words = word_tokenize(string_script_text)
#words

In [14]:
sentences = sent_tokenize(string_script_text)
#sentences

In [15]:
dialogue = []

for sentence in sentences:
    if sentence.split(':')[0] in (names7_01):
        dialogue.append(sentence)

In [16]:
#dialogue

In [17]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s701.pkl', 'wb') as f:
    pickle.dump(dialogue, f)

In [18]:
from collections import defaultdict

speakers = defaultdict(list)
for sentence in dialogue:
    speakers[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers

In [19]:
speakers['ARYA']

[u' When people ask you what happened here, tell them the North remembers.',
 u" That's a pretty song.",
 u" I don't want to steal your food.",
 u" King's Landing.",
 u' Not so bad, is it?',
 u' What are you doing in the Riverlands?',
 u" Oh, no, I couldn't.",
 u" You're a long way from home.",
 u' Boy or girl?',
 u' Why?',
 u" It's really good.",
 u" I'm going to kill the queen."]

In [20]:
episode_one = pd.DataFrame.from_dict(speakers, orient="index")
print episode_one.shape

(30, 30)


In [21]:
#from numpy import nan
#episode_one.fillna(value=str(0), inplace=True)

In [22]:
#episode_one.info()

In [23]:
#episode_one.head()

In [24]:
def remove_stops(sentence):
    characters = ['?','!','.',',',':',';']
    stoplist = stopwords.words('english') + characters
    try: 
        new_sentence = re.sub(r'[^a-zA-Z ]','', sentence)
        new_sentence = new_sentence.lower()
        new_sentence = new_sentence.split()
        new_sentence = [word for word in new_sentence if word not in stoplist]
        new_sentence = ' '.join(new_sentence)
        return new_sentence
    except:
        return sentence   

In [25]:
test = episode_one[0].apply(remove_stops)
#test

In [26]:
for column in episode_one.columns:
    for x in range(len(episode_one[column])):
        episode_one[column][x] = remove_stops(episode_one[column][x])

In [27]:
episode_one.index = episode_one.index.str.title()

In [28]:
episode_one.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Yohn,umbers karstarks betrayed north,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Gilly,really sleep sam,legends long night,,thats dragonglass,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,,knew,angry,,dragonstone,deepwater ports ships,winter,think daenerys win,three kingdoms best,dynasty,never talked tommen,baby boy killed,cersei jaime,couldnt,well looks like,better allies,theyre good anything,oh come,youre rightful monarch though,greyjoys rebelled agains throne right monarchs...,people cutting kin,enjoyed killing greyjoys,None,None,None,None,None,None,None,None
Euron,nothing compares treason suffered hands family...,remember well,place getting crowded,good thing,youre humble,ever since little boy wanted grow mary beautif...,,try,dont expect trust outright,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sam,archmaester wondering marwyn,um,wondering considered proposition,asked light ive seen north could access restri...,,archmaester respect ive seen themthe army dead,sources restricted area,believe,makes person south twins,dead dont,targaryens used dragonglass decorate weapons w...,map dragonstone,mountain,,havent heard anything,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Two

In [29]:
#df_s7.script_text[1]

In [30]:
script_text7_02 = df_s7.script_text[1]
print type(script_text7_02)

#script_text7_02

<type 'list'>


In [31]:
script_split2 = [i.split('\n') for i in script_text7_02]
#script_split2

In [32]:
string_script_text2 = ' '

for i in script_split2:
    for e in i:
        string_script_text2 = string_script_text2 + ' ' + e

In [33]:
#string_script_text2

In [34]:
words2 = word_tokenize(string_script_text2)
#words2

In [35]:
sentences2 = sent_tokenize(string_script_text2)
#sentences2

In [36]:
dialogue2 = []

for sentence in sentences2:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names7_02):
            dialogue2.append(sentence)

In [37]:
#dialogue2

In [38]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s702.pkl', 'wb') as f:
    pickle.dump(dialogue2, f)

In [39]:
from collections import defaultdict

speakers2 = defaultdict(list)
for sentence in dialogue2:
    speakers2[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers2

In [40]:
speakers2['TYRION']

[u" We won't stay on Dragonstone for long.",
 u' Conquering Westeros would be easy for you.',
 u' We can take the Seven Kingdoms without turning it into a slaughterhouse.',
 u' Your Grace, when I was ready to drink myself into a small coffin, Lord Varys told me about a queen in the east who--    DAENERYS (to VARYS)',
 u' Lord Varys has proven himself a loyal servant.',
 u"  Doesn't really roll off the tongue, does it?",
 u'  Jon Snow?',
 u"  I traveled with him to the Wall when he joined the Night's Watch.",
 u"  I can't speak to prophecies or visions in the flames, but I like Jon Snow and I trusted him, and I am an excellent judge of character.",
 u'  If he does rule the north, he would make a valuable ally.',
 u'  If we turn the dragons loose, tens of thousands will die in the firestorms.',
 u'    I know how you wage war.',
 u"  But we won't use Dothraki and Unsullied.",
 u'  Cersei will try to rally the lords of Westeros by appealing to their loyalty, their love for their country.',

In [41]:
episode_two = pd.DataFrame.from_dict(speakers2, orient="index")
print episode_two.shape

(35, 37)


In [42]:
#from numpy import nan
#episode_two.fillna(value=str(0), inplace=True)

In [43]:
#episode_two.head()

In [44]:
test2 = episode_two[0].apply(remove_stops)
#test2

In [45]:
for column in episode_two.columns:
    for x in range(len(episode_two[column])):
        episode_two[column][x] = remove_stops(episode_two[column][x])

In [46]:
episode_two.index = episode_two.index.str.title()

In [47]:
episode_two.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
Davos,sounds like charmer,fire kills whites told,dangerous,army dead makes passed wall enough men fight,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Melisandre,called melisandre,didnt,long night coming,prophecies dangerous things,lord commander nights watch allowed wildlings ...,summon jon snow,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Dickon,ser jaime,dickon,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyene,momma want cersei,father got careless,maybe ill kill take kings landing wont share,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,lord tarley,rickon isnt,thats,thank coming,ride hornhill today,side fighting,want ranking general wars come,swore oath crown well lord tarley,great woman,war queen need new warden south,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Three

In [48]:
#df_s7.script_text[2]

In [49]:
script_text7_03 = df_s7.script_text[2]
print type(script_text7_03)

#script_text7_03

<type 'list'>


In [50]:
script_split3 = [i.split('\n') for i in script_text7_03]
#script_split3

In [51]:
string_script_text3 = ' '

for i in script_split3:
    for e in i:
        string_script_text3 = string_script_text3 + ' ' + e

In [52]:
#string_script_text3

In [53]:
words3 = word_tokenize(string_script_text3)
#words3

In [54]:
sentences3 = sent_tokenize(string_script_text3)
#sentences3

In [55]:
dialogue3 = []

for sentence in sentences3:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names7_03):
            dialogue3.append(sentence)

In [56]:
#dialogue3

In [57]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s703.pkl', 'wb') as f:
    pickle.dump(dialogue3, f)

In [58]:
from collections import defaultdict

speakers3 = defaultdict(list)
for sentence in dialogue3:
    speakers3[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers3

In [59]:
speakers3['BRAN']

[u'  Hello, Sansa.',
 u'  Yes.',
 u'  I can never be the Lord of Winterfell.',
 u"  It's difficult to explain.",
 u'  It means I can see everything.',
 u'  The three-eyed raven taught me.',
 u"  I told you it's difficult to explain.",
 u"  I'll stay a bit longer."]

In [60]:
episode_three = pd.DataFrame.from_dict(speakers3, orient="index")
print episode_three.shape

(26, 50)


In [61]:
#from numpy import nan
#episode_three.fillna(value=str(0), inplace=True)

In [62]:
#episode_three.head()

In [63]:
test3 = episode_three[0].apply(remove_stops)
#test3

In [64]:
for column in episode_three.columns:
    for x in range(len(episode_three[column])):
        episode_three[column][x] = remove_stops(episode_three[column][x])

In [65]:
episode_three.index = episode_three.index.str.title()

In [66]:
episode_three.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
Davos,davos seaworth,unluckily,,ah,place changed,jon snow,hes king north,apologies,wasnt grace,defeat cersei,dont believe,destiny brough daenerys targaryen back shores ...,things dont believe faced things,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Yohn,current occupants castle enough food year perhaps,wise lady,lady,indeed,isnt leather,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Melisandre,ive done part,time whispering ears kings come end,neither us common folk anymore,mistakes made,volantis,return dear spider,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,mob spat sister long ago turn us theyll cheer ...,advice,dont,,,suppose well expected,truth casterly rock isnt worth much anymore,,robb stark whispering wood,father always said slow learner,widows wail,sure,possibly,yes,think done,cersei several ideas,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Messenger,forgive grace,grace,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Four

In [67]:
#df_s7.script_text[3]

In [68]:
script_text7_04 = df_s7.script_text[3]
print type(script_text7_04)

#script_text7_04

<type 'list'>


In [69]:
script_split4 = [i.split('\n') for i in script_text7_04]
#script_split4

In [70]:
string_script_text4 = ' '

for i in script_split4:
    for e in i:
        string_script_text4 = string_script_text4 + ' ' + e

In [71]:
#string_script_text4

In [72]:
words4 = word_tokenize(string_script_text4)
#words4

In [73]:
sentences4 = sent_tokenize(string_script_text4)
#sentences4

In [74]:
dialogue4 = []

for sentence in sentences4:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names7_04):
            dialogue4.append(sentence)

In [75]:
#dialogue4

In [76]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s704.pkl', 'wb') as f:
    pickle.dump(dialogue4, f)

In [77]:
from collections import defaultdict

speakers4 = defaultdict(list)
for sentence in dialogue4:
    speakers4[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers4

In [78]:
speakers4['ARYA']

[u'  In there.',
 u"  I'm Arya Stark.",
 u'  Send for Maester Llewyn or Ser Rodrick.',
 u'  Go ask Jon Snow then, the King in the North.',
 u'  If Jon Snow is gone who is in charge of Winterfell?',
 u'  Which Lady Stark?',
 u'  Tell Sansa her sister is home.',
 u"  I'm getting into this castle one way or another.",
 u'  Do I have to call you Lady Stark now?',
 u"  I didn't run.",
 u'  It suits you, Lady Stark.',
 u"  That doesn't look like him.",
 u"  We're not.",
 u'  They say you killed Joffrey.',
 u'  Me too.',
 u"  Of people I'm going to kill.",
 u"  It's a long story.",
 u"  Mine neither, but our stories aren't over yet.",
 u'  You saw me?',
 u'  So did I.    SANSA',
 u'  Most of them are dead already.',
 u'  Where did you get this?',
 u"  Littlefinger, he's here?",
 u'  Why would a cutthroat have a Valyrian steel dagger?',
 u'  Are you sure?',
 u"  He didn't beat the hound.",
 u"  You swore to serve both of my mother's daughters, didn't you?",
 u"  I won't cut you.",
 u'  No one.

In [79]:
episode_four = pd.DataFrame.from_dict(speakers4, orient="index")
print episode_four.shape

(21, 32)


In [80]:
#from numpy import nan
#episode_four.fillna(value=str(0), inplace=True)

In [81]:
#episode_four.head()

In [82]:
test4 = episode_four[0].apply(remove_stops)
#test4

In [83]:
for column in episode_four.columns:
    for x in range(len(episode_four[column])):
        episode_four[column][x] = remove_stops(episode_four[column][x])

In [84]:
episode_four.index = episode_four.index.str.title()

In [85]:
episode_four.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
Randyll,granaries emptied loaded wagons lord,lord,gold safely gates kings landing,need get last wagons blackwater rush nightfall,permission flogging stragglers marked effect m...,come lads,spears armed,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Davos,youll want discuss amongst,think,good heart,fewer,speaking good hearts missandei naarth,king snow isnt,custom different naarth,sounds liberating,may slave girl come advise daenerys targaryen,good,heard uncle attacked fleet,sister,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Baelish,,last man wielded meant cut throat mother fought,dagger one took life would stopped dagger hear...,,go make way home find chaos world imagine bran...,im sorry disturbed lord stark,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion,took casterly rock,still largest army,call grey worm unsullied back,ive underestimated enemies daenerys,weve discussed,smart one,flee idiot,fucking idiot,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Bronn,biggest prize world,high septon,still question prize,castle,one,beg differ,oh youre fucking glum eh,right,yes im sure queen cerseis reign quiet peaceful,like,im much shoveling weight,come,men shit die,listen,line,get back kings landing,youre commander damned infantryman,go get,,come fucker,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Five

In [86]:
#df_s7.script_text[4]

In [87]:
script_text7_05 = df_s7.script_text[4]
print type(script_text7_05)

#script_text7_05

<type 'list'>


In [88]:
script_split5 = [i.split('\n') for i in script_text7_05]
#script_split5

In [89]:
string_script_text5 = ' '

for i in script_split5:
    for e in i:
        string_script_text5 = string_script_text5 + ' ' + e

In [90]:
#string_script_text5

In [91]:
words5 = word_tokenize(string_script_text5)
#words5

In [92]:
sentences5 = sent_tokenize(string_script_text5)
#sentences5

In [93]:
dialogue5 = []

for sentence in sentences5:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names7_05):
            dialogue5.append(sentence)

In [94]:
#dialogue5

In [95]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s705.pkl', 'wb') as f:
    pickle.dump(dialogue5, f)

In [96]:
from collections import defaultdict

speakers5 = defaultdict(list)
for sentence in dialogue5:
    speakers5[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers5

In [97]:
speakers5['GENDRY']

[u"  Weren't you worried about the Goldcloaks?",
 u'  Here I am arming Lannisters.',
 u'  You came to get me.',
 u"  Don't you think I've been thinking about it every swing of the hammer... how happy I am making weapons for the family that killed my father?",
 u"  I've been getting ready.",
 u"  I don't know much about swinging swords, but this --    GENDRY removes a giant war hammer from the wall.",
 u'  -- this I know.',
 u'  Who should I say I am?',
 u'  That they are.',
 u'  I understand.',
 u"  You don't have to worry.",
 u'  Understood.',
 u"  It's Gendry, Your Grace.",
 u'  Our fathers trusted each other.',
 u'  I met yours in my shop.',
 u'  All I ever knew was they fought together and won.',
 u"  No, but I'm a fighter.",
 u'  I prefer a hammer.',
 u"  Don't trust them.",
 u"  They're the Brotherhood.",
 u'  How can we be?']

In [98]:
episode_five = pd.DataFrame.from_dict(speakers5, orient="index")
print episode_five.shape

(32, 44)


In [99]:
#from numpy import nan
#episode_five.fillna(value=str(0), inplace=True)

In [100]:
#episode_five.head()

In [101]:
test5 = episode_five[0].apply(remove_stops)
#test5

In [102]:
for column in episode_five.columns:
    for x in range(len(episode_five[column])):
        episode_five[column][x] = remove_stops(episode_five[column][x])

In [103]:
episode_five.index = episode_five.index.str.title()

In [104]:
episode_five.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
Davos,smuggle goldcloaks recognize im warning im fig...,wont follow ser jorah,cant lead raid beyond wall,last time killed son wildfire,theres path left hugs cliff,ive got business flea bottom,fucked,wasnt sure id find,thought might still rowing,looked shops taverns brothels,havent years,anybody give trouble,dont sure,think need understand gendry,know youre heading,might want bring one swords,keep fathers name,told clovis dont get soon someone going guard,weapons friends,,still five gold dragons,,cant say pleasure business,gold,come look,fermented crab,fermented crab,id hurry favorite establishment youll put whol...,really wouldnt try anymore see first bite hits,perhaps theres arrangement,gendry,leaving today,doesnt need thinking harboring bastard dead king,youre clovis smith whos come pay respects head...,begging pardon grace,yet,meant keep,dont fool,handle,father used say better coward minute dead rest...,yeah,ive failing job late,ill staying behind,reasons arent reasons
Yohn,knights vale came lady stark,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Dickon,kill,im dickon tarley son randyll tarley,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Gilly,know many steps citadel,,guess many windows great sept baelor,thats true,meynard says issued anullment prince rhaegar r...,steps,sure want,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,ending war killing,one,dont mean fucked,tell cersei,havent done full counting,saw dothraki fight,didnt,im talking father,,olenna would rather seen granddaughter married...,shes dead like son grandchildren whole house,dont time,seem recall giving bit trouble well,,today might day kill accident,dont talk father,want cersei bend knee ask,qyburn,met tyrion,daenerys wants meet,discuss armistice,army dead men marching seven kingdoms,tyrion,let happen,say father,people wont like,lion doesnt concern opinions sheep,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Six

In [105]:
#df_s7.script_text[5]

In [106]:
script_text7_06 = df_s7.script_text[5]
print type(script_text7_06)

#script_text7_06

<type 'list'>


In [107]:
script_split6 = [i.split('\n') for i in script_text7_06]
#script_split6

In [108]:
string_script_text6 = ' '

for i in script_split6:
    for e in i:
        string_script_text6 = string_script_text6 + ' ' + e

In [109]:
#string_script_text6

In [110]:
words6 = word_tokenize(string_script_text6)
#words6

In [111]:
sentences6 = sent_tokenize(string_script_text6)
#sentences6

In [112]:
dialogue6 = []

for sentence in sentences6:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names7_06):
            dialogue6.append(sentence)

In [113]:
#dialogue6

In [114]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s706.pkl', 'wb') as f:
    pickle.dump(dialogue6, f)

In [115]:
from collections import defaultdict

speakers6 = defaultdict(list)
for sentence in dialogue6:
    speakers6[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers6

In [116]:
speakers6['ARYA']

[u'  Father used to watch us from up here.',
 u'  One time the boys were shooting arrows with Ser Rodrick.',
 u"  I looked up and he's standing right here smiling down at me.",
 u"  That's your pretty handwriting.",
 u'  "Father has been charged with treason.',
 u'  Did they.',
 u'  So was I.  I would have let them kill me before I betrayed my family.',
 u'  And you were stupid enough to believe them.',
 u"  I was there standing in the crowd near Baelor's statue.",
 u'  I wanted to.',
 u"  I didn't betray him.",
 u'  I was training.',
 u"  Oh, I don't know about that.",
 u"  I guess we'll never know.",
 u"  I don't know yet.",
 u"  You're scared, aren't you?",
 u'  Sometimes fear makes them do unfortunate things.',
 u" Not what you're looking for?",
 u" They're not here now.",
 u' My faces.',
 u' In Braavos, while I was training to be a Faceless Man.',
 u' Back in Braavos, before I got my first face, there was a game I used to play.',
 u' How do you feel about Jon being king?',
 u' ou 

In [117]:
episode_six = pd.DataFrame.from_dict(speakers6, orient="index")
print episode_six.shape

(14, 41)


In [118]:
#from numpy import nan
#episode_six.fillna(value=str(0), inplace=True)

In [119]:
#episode_six.head()

In [120]:
test6 = episode_six[0].apply(remove_stops)
#test6

In [121]:
for column in episode_six.columns:
    for x in range(len(episode_six[column])):
        episode_six[column][x] = remove_stops(episode_six[column][x])

In [122]:
episode_six.index = episode_six.index.str.title()

In [123]:
episode_six.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Benjen,ride pass,theres time,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sansa,remember,,forced,dont know like,told way save father,,,didnt,knees thanking,training,never would survived survived,letter,show,arya arya,know happy cersei would saw us fighting,youre angry,get,asking twenty thousand men fight us worst wint...,loyalty jon,yes turned backs jon time retake winterfell na...,would thought going betray jon,dont know thinks,baelish,would jorah,invitation kings landing,represent interests gathering see,set foot kings landing cersei lannister queen,ser jaime,many guards would happily imprison behead whet...,hundreds men winterfell loyal,,get,mean,dont want play,faces,tell,None,None,None,None,None
Gendry,never seen snow,sold witch,wanted one,put leeches,yes,wanted kill,im whinging,bears blue eyes,im leaving,raven need send raven,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Brienne,lady lady winterfell,invited,safe,im worried,trust loyalty,lady swore oath protect sister,yes lady,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Baelish,dont know,question loyalty,youre lady winterfell,aryas like,thinks,perhaps lady brienne could help,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Season Seven

In [124]:
episode_one.index

Index([u'Yohn', u'Gilly', u'Jaime', u'Euron', u'Sam', u'Walder Frey', u'Meera',
       u'Daenerys', u'Marwyn', u'Jon', u'Thoros', u'Cersei', u'Beric',
       u'Walder', u'Petyr', u'Robett', u'Lyanna', u'All', u'Edd', u'Sansa',
       u'Brienne', u'Man #2', u'Wolkan', u'Jorah', u'Man #3', u'Sandor',
       u'Tormund', u'Arya', u'Bran', u'Man'],
      dtype='object')

In [125]:
episode_one = episode_one.reset_index()

In [126]:
episode_one_dict = {'Yohn':'Yohn Royce',
                    'Gilly':'Gilly',
                    'Jaime':'Jaime',
                    'Euron':'Euron',
                    'Sam':'Sam',
                    'Walder Frey':'Walder Frey',
                    'Meera':'Meera',
                    'Marwyn':'Marwyn',
                    'Jon':'Jon',
                    'Thoros':'Thoros',
                    'Cersei':'Cersei',
                    'Beric':'Beric',
                    'Walder':'Walder Frey',
                    'Petyr':'Baelish',
                    'Robett':'Robett Glover',
                    'Lyanna':'Lyanna Mormont',
                    'All':'All',
                    'Edd':'Edd',
                    'Sansa':'Sansa',
                    'Brienne':'Brienne',
                    'Man #2':'Man #2',
                    'Wolkan':'Wolkan',
                    'Jorah':'Jorah',
                    'Man #3':'Man #3',
                    'Sandor':'Sandor',
                    'Tormund':'Tormund', 
                    'Arya':'Arya', 
                    'Bran':'Bran',
                    'Man':'Man'}

episode_one['index'] = episode_one['index'].replace(episode_one_dict)

In [127]:
episode_one = episode_one.set_index('index')

In [128]:
episode_two.index

Index([u'Davos', u'Melisandre', u'Dickon', u'Tyene', u'Jaime', u'Hot Pie',
       u'Ellia', u'Tyrion', u'Euron', u'Missandei', u'Yara', u'Baelish',
       u'Ellaria', u'Robett Glover', u'Olenna', u'Samwell', u'Daenerys',
       u'Marwyn', u'Jon', u'Theon', u'Lyanna Mormont', u'Grey Worm', u'Cersei',
       u'Varys', u'Qyburn', u'Nymeria', u'Randyll', u'Obara', u'Sansa',
       u'Man #2', u'Man #1', u'Jorah', u'Yohn Royce', u'Arya', u'Wolkan'],
      dtype='object')

In [129]:
episode_two = episode_two.reset_index()

In [130]:
episode_two_dict = {'Davos':'Davos', 
                    'Melisandre':'Melisandre', 
                    'Dickon':'Dickon', 
                    'Tyene':'Tyene',
                    'Yohn Royce':'Yohn Royce',
                    'Jaime':'Jaime',
                    'Hot Pie':'Hot Pie',
                    'Ellia':'Ellaria',
                    'Tyrion':'Tyrion',
                    'Olenna':'Olenna',
                    'Euron':'Euron',
                    'Missandei':'Missandei',
                    'Yara':'Yara',
                    'Ellaria':'Ellaria',
                    'Samwell':'Sam',
                    'Daenerys':'Daenerys',
                    'Marwyn':'Marwyn',
                    'Jon':'Jon',
                    'Theon':'Theon',
                    'Cersei':'Cersei',
                    'Varys':'Varys',
                    'Qyburn':'Qyburn',
                    'Nymeria':'Nymeria', 
                    'Randyll':'Randyll', 
                    'Obara':'Obara',
                    'Baelish':'Baelish',
                    'Robett Glover':'Robett Glover',
                    'Lyanna Mormont':'Lyanna Mormont',
                    'Grey Worm':'Grey Worm',
                    'Sansa':'Sansa',
                    'Man #2':'Man #2',
                    'Wolkan':'Wolkan',
                    'Jorah':'Jorah',
                    'Man #1':'Man #1', 
                    'Arya':'Arya'}

episode_two['index'] = episode_two['index'].replace(episode_two_dict)

In [131]:
episode_two = episode_two.set_index('index')

In [132]:
print episode_one.shape
print episode_two.shape

(30, 30)
(35, 37)


In [133]:
season_seven = episode_one.join(episode_two, rsuffix="_two", how='outer')
print season_seven.shape

(49, 67)


In [134]:
#season_seven.index

In [135]:
episode_three.index

Index([u'Davos', u'Yohn', u'Melisandre', u'Jaime', u'Messenger', u'Tyrion',
       u'Euron', u'Missandei', u'Baelish', u'Olenna', u'Samwell', u'Guard',
       u'Daenerys', u'Marwyn', u'Jon', u'Theon', u'Grey Worm', u'Cersei',
       u'Varys', u'Qyburn', u'Sansa', u'Wolkan', u'Jorah', u'Tycho',
       u'Captain', u'Bran'],
      dtype='object')

In [136]:
episode_three = episode_three.reset_index()

In [137]:
episode_three_dict = {'Davos':'Davos', 
                    'Melisandre':'Melisandre',
                    'Yohn':'Yohn Royce',
                    'Jaime':'Jaime',
                    'Messenger':'Messenger',
                    'Tyrion':'Tyrion',
                    'Olenna':'Olenna',
                    'Euron':'Euron',
                    'Missandei':'Missandei',
                    'Samwell':'Sam',
                    'Guard':'Guard',
                    'Daenerys':'Daenerys',
                    'Marwyn':'Marwyn',
                    'Jon':'Jon',
                    'Theon':'Theon',
                    'Cersei':'Cersei',
                    'Varys':'Varys',
                    'Qyburn':'Qyburn',
                    'Baelish':'Baelish',
                    'Grey Worm':'Grey Worm',
                    'Sansa':'Sansa',
                    'Wolkan':'Wolkan',
                    'Jorah':'Jorah', 
                    'Arya':'Arya',
                    'Tycho':'Tycho',
                    'Bran':'Bran'}

episode_three['index'] = episode_three['index'].replace(episode_three_dict)

In [138]:
episode_three = episode_three.set_index('index')

In [139]:
episode_three.shape

(26, 50)

In [140]:
season_seven = season_seven.join(episode_three, rsuffix="_three", how='outer')
print season_seven.shape

(53, 117)


In [141]:
#season_seven.index

In [142]:
episode_four.index

Index([u'Randyll', u'Davos', u'Baelish', u'Tyrion', u'Bronn', u'Sansa',
       u'Man 1', u'Man 2', u'Cersei', u'Tycho', u'Brienne', u'Jaime',
       u'Dickon', u'Podrick', u'Daenerys', u'Theon', u'Arya', u'Missandei',
       u'Bran', u'Jon', u'Meera'],
      dtype='object')

In [143]:
episode_four.shape

(21, 32)

In [144]:
season_seven = season_seven.join(episode_four, rsuffix="_four", how='outer')
print season_seven.shape

(57, 149)


In [145]:
#season_seven.index

In [146]:
episode_five.index

Index([u'Davos', u'Yohn', u'Dickon', u'Gilly', u'Jaime', u'Tyrion', u'Baelish',
       u'Sam', u'Gendry', u'Samwell', u'Guard', u'Daenerys', u'Guard 2',
       u'Marwyn', u'Jon', u'Thoros', u'Beric', u'Maester 2', u'Maester 1',
       u'Cersei', u'Varys', u'Bronn', u'Guard 1', u'Robett', u'Qyburn',
       u'Randyll', u'Sansa', u'Wolkan', u'Jorah', u'Tormund', u'Arya',
       u'Bran'],
      dtype='object')

In [147]:
episode_five = episode_five.reset_index()

In [148]:
episode_five_dict = {'Davos':'Davos',
                     'Yohn':'Yohn Royce', 
                     'Dickon':'Dickon', 
                     'Gilly':'Gilly', 
                     'Jaime':'Jaime', 
                     'Tyrion':'Tyrion', 
                     'Baelish':'Baelish',
                     'Sam':'Sam', 
                     'Gendry':'Gendry', 
                     'Samwell':'Samwell', 
                     'Guard':'Guard', 
                     'Daenerys':'Daenerys', 
                     'Guard 2':'Guard 2',
                     'Marwyn':'Marwyn', 
                     'Jon':'Jon', 
                     'Thoros':'Thoros', 
                     'Beric':'Beric', 
                     'Maester 2':'Maester 2', 
                     'Maester 1':'Maester 1',
                     'Cersei':'Cersei', 
                     'Varys':'Varys', 
                     'Bronn':'Bronn', 
                     'Guard 1':'Guard 1', 
                     'Robett':'Robett Glover', 
                     'Qyburn':'Qyburn',
                     'Randyll':'Randyll', 
                     'Sansa':'Sansa', 
                     'Wolkan':'Wolkan', 
                     'Jorah':'Jorah', 
                     'Tormund':'Tormund', 
                     'Arya':'Arya',
                     'Bran':'Bran'}

episode_five['index'] = episode_five['index'].replace(episode_five_dict)

In [149]:
episode_five = episode_five.set_index('index')

In [150]:
episode_five.shape

(32, 44)

In [151]:
season_seven = season_seven.join(episode_five, rsuffix="_five", how='outer')
print season_seven.shape    

(63, 193)


In [152]:
#season_seven.index

In [153]:
episode_six.index

Index([u'Benjen', u'Sansa', u'Gendry', u'Brienne', u'Baelish', u'Jorah',
       u'Beric', u'Sandor', u'Tormund', u'Tyrion', u'Daenerys', u'Arya',
       u'Jon', u'Thoros'],
      dtype='object')

In [154]:
episode_six.shape

(14, 41)

In [155]:
season_seven = season_seven.join(episode_six, rsuffix="_six", how='outer')
print season_seven.shape    

(64, 234)


In [156]:
#season_seven.index

In [157]:
from numpy import nan
season_seven.fillna(value=str(0), inplace=True)

In [158]:
#season_seven.head(65)

In [159]:
season_seven.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, All to Yohn Royce
Columns: 234 entries, 0 to 40
dtypes: object(234)
memory usage: 117.5+ KB


In [160]:
season_seven.to_pickle('/Users/zoemarkovits/Metis/project_five_got/PKLs/season_seven.pkl')

## Character Breakdown

### Arya Stark

In [161]:
Arya = season_seven.loc['Arya']
#print Arya

In [162]:
Arya_Text = Arya.values.tolist()
#Arya_Text

In [163]:
Arya_Words = []

for word in Arya_Text:
    if len(word)>=1:
        if word != '0':
            Arya_Words.append(word)
        
#Arya_Words 

In [164]:
Arya_Words_String = ' '.join(Arya_Words)
#Arya_Words_String

In [165]:
Arya_String_Length = len(Arya_Words_String)
Arya_String_Length

1571

In [166]:
Arya_Sentiment = TextBlob(Arya_Words_String).sentiment
Arya_Sentiment

Sentiment(polarity=0.02489795918367348, subjectivity=0.5426757369614511)

### Brienne of Tarth

In [167]:
Brienne = season_seven.loc['Brienne']
#print Brienne

In [169]:
Brienne_Text = Brienne.values.tolist()
#Brienne_Text

In [170]:
Brienne_Words = []

for word in Brienne_Text:
    if len(word)>=1:
        if word != '0':
            Brienne_Words.append(word)
        
#Brienne_Words 

In [171]:
Brienne_Words_String = ' '.join(Brienne_Words)
#Brienne_Words_String

In [172]:
Brienne_String_Length = len(Brienne_Words_String)
Brienne_String_Length

302

In [173]:
Brienne_Sentiment = TextBlob(Brienne_Words_String).sentiment
Brienne_Sentiment

Sentiment(polarity=0.16, subjectivity=0.52)

### Cersei Lannister

In [174]:
Cersei = season_seven.loc['Cersei']
#print Cersei

In [175]:
Cersei_Text = Cersei.values.tolist()
#Cersei_Text

In [176]:
Cersei_Words = []

for word in Cersei_Text:
    if len(word)>=1:
        if word != '0':
            Cersei_Words.append(word)
        
#Cersei_Words 

In [177]:
Cersei_Words_String = ' '.join(Cersei_Words)
#Cersei_Words_String

In [178]:
Cersei_String_Length = len(Cersei_Words_String)
Cersei_String_Length

2089

In [179]:
Cersei_Sentiment = TextBlob(Cersei_Words_String).sentiment
Cersei_Sentiment

Sentiment(polarity=0.2639994414187963, subjectivity=0.591339663920309)

### Daenerys Targaryen

In [180]:
Daenerys = season_seven.loc['Daenerys']
#print Daenerys

In [181]:
Dany_Text = Daenerys.values.tolist()
#Dany_Text

In [182]:
Dany_Words = []

for word in Dany_Text:
    if len(word)>=1:
        if word != '0':
            Dany_Words.append(word)
        
#Dany_Words 

In [184]:
Dany_Words_String = ' '.join(Dany_Words)
#Dany_Words_String

In [185]:
Dany_String_Length = len(Dany_Words_String)
Dany_String_Length

3507

In [186]:
Dany_Sentiment = TextBlob(Dany_Words_String).sentiment
Dany_Sentiment

Sentiment(polarity=0.17328869047619053, subjectivity=0.5563461061507936)

### Davos

In [187]:
Davos = season_seven.loc['Davos']
#print Davos

In [189]:
Davos_Text = Davos.values.tolist()
#Davos_Text

In [190]:
Davos_Words = []

for word in Davos_Text:
    if len(word)>=1:
        if word != '0':
            Davos_Words.append(word)
        
#Davos_Words 

In [191]:
Davos_Words_String = ' '.join(Davos_Words)
#Davos_Words_String

In [192]:
Davos_String_Length = len(Davos_Words_String)
Davos_String_Length

1562

In [193]:
Davos_Sentiment = TextBlob(Davos_Words_String).sentiment
Davos_Sentiment

Sentiment(polarity=0.053999999999999986, subjectivity=0.4991111111111111)

### Gilly

In [194]:
Gilly = season_seven.loc['Gilly']
#print Gilly

In [195]:
Gilly_Text = Gilly.values.tolist()
#Gilly_Text

In [196]:
Gilly_Words = []

for word in Gilly_Text:
    if len(word)>=1:
        if word != '0':
            Gilly_Words.append(word)
        
#Gilly_Words 

In [198]:
Gilly_Words_String = ' '.join(Gilly_Words)
#Gilly_Words_String

In [199]:
Gilly_String_Length = len(Gilly_Words_String)
Gilly_String_Length

236

In [200]:
Gilly_Sentiment = TextBlob(Gilly_Words_String).sentiment
Gilly_Sentiment

Sentiment(polarity=0.3, subjectivity=0.5736111111111111)

### Jaime Lannister

In [201]:
Jaime = season_seven.loc['Jaime']
#print Jaime

In [202]:
Jaime_Text = Jaime.values.tolist()
#Jaime_Text

In [203]:
Jaime_Words = []

for word in Jaime_Text:
    if len(word)>=1:
        if word != '0':
            Jaime_Words.append(word)
        
#Jaime_Words 

In [204]:
Jaime_Words_String = ' '.join(Jaime_Words)
#Jaime_Words_String

In [205]:
Jaime_String_Length = len(Jaime_Words_String)
Jaime_String_Length

1953

In [206]:
Jaime_Sentiment = TextBlob(Jaime_Words_String).sentiment
Jaime_Sentiment

Sentiment(polarity=0.11179684445988791, subjectivity=0.5034611463415811)

### Jon Snow

In [207]:
Jon = season_seven.loc['Jon']
#print Jon

In [208]:
Jon_Text = Jon.values.tolist()
#Jon_Text

In [209]:
Jon_Words = []

for word in Jon_Text:
    if len(word)>=1:
        if word != '0':
            Jon_Words.append(word)
        
#Jon_Words 

In [210]:
Jon_Words_String = ' '.join(Jon_Words)
#Jon_Words_String

In [211]:
Jon_String_Length = len(Jon_Words_String)
Jon_String_Length

3899

In [212]:
Jon_Sentiment = TextBlob(Jon_Words_String).sentiment
Jon_Sentiment

Sentiment(polarity=0.07213478611783697, subjectivity=0.473547215496368)

### Jorah Mormont

In [213]:
Jorah = season_seven.loc['Jorah']
#print Jorah

In [214]:
Jorah_Text = Jorah.values.tolist()
#Jorah_Text

In [215]:
Jorah_Words = []

for word in Jorah_Text:
    if len(word)>=1:
        if word != '0':
            Jorah_Words.append(word)
        
#Jorah_Words 

In [216]:
Jorah_Words_String = ' '.join(Jorah_Words)
#Jorah_Words_String

In [217]:
Jorah_String_Length = len(Jorah_Words_String)
Jorah_String_Length

609

In [218]:
Jorah_Sentiment = TextBlob(Jorah_Words_String).sentiment
Jorah_Sentiment

Sentiment(polarity=-0.03358843537414965, subjectivity=0.41284013605442166)

### Petyr Baelish/Littlefinger

In [219]:
Baelish = season_seven.loc['Baelish']

In [220]:
Baelish_Text = Baelish.values.tolist()

In [221]:
Baelish_Words = []

for word in Baelish_Text:
    if len(word)>=1:
        if word != '0':
            Baelish_Words.append(word)           
            
#Baelish_Words 

In [222]:
Baelish_Words_String = ' '.join(Baelish_Words)
#Baelish_Words_String

In [223]:
Baelish_String_Length = len(Baelish_Words_String)
Baelish_String_Length

533

In [224]:
Baelish_Sentiment = TextBlob(Baelish_Words_String).sentiment
Baelish_Sentiment

Sentiment(polarity=0.12077922077922079, subjectivity=0.6427128427128428)

### Melisandre

In [225]:
Melisandre = season_seven.loc['Melisandre']
#print Melisandre

In [226]:
Melisandre_Text = Melisandre.values.tolist()
#Melisandre_Text

In [227]:
Melisandre_Words = []

for word in Melisandre_Text:
    if len(word)>=1:
        if word != '0':
            Melisandre_Words.append(word)
        
#Melisandre_Words 

In [228]:
Melisandre_Words_String = ' '.join(Melisandre_Words)
#Melisandre_Words_String

In [229]:
Melisandre_String_Length = len(Melisandre_Words_String)
Melisandre_String_Length

286

In [230]:
Melisandre_Sentiment = TextBlob(Melisandre_Words_String).sentiment
Melisandre_Sentiment

Sentiment(polarity=-0.03749999999999999, subjectivity=0.6375)

### Missandei

In [231]:
Missandei = season_seven.loc['Missandei']
#print Missandei

In [232]:
type(Missandei)

pandas.core.series.Series

In [233]:
Missandei_Text = Missandei.values.tolist()
#Missandei_Text

In [234]:
Missandei_Words = []

for word in Missandei_Text:
    if len(word)>=1:
        if word != '0':
            Missandei_Words.append(word)
        
#Missandei_Words 

In [235]:
Missandei_Words_String = ' '.join(Missandei_Words)
#Missandei_Words_String

In [236]:
Missandei_String_Length = len(Missandei_Words_String)
Missandei_String_Length

444

In [237]:
Missandei_Sentiment = TextBlob(Missandei_Words_String).sentiment
Missandei_Sentiment

Sentiment(polarity=0.4428571428571429, subjectivity=0.6476190476190476)

### Olenna Tyrell

In [238]:
Olenna = season_seven.loc['Olenna']

In [239]:
Olenna_Text = Olenna.values.tolist()

In [240]:
Olenna_Words = []

for word in Olenna_Text:
    if len(word)>=1:
        if word != '0':
            Olenna_Words.append(word)           
            
#Olenna_Words 

In [241]:
Olenna_Words_String = ' '.join(Olenna_Words)
#Olenna_Words_String

In [242]:
Olenna_String_Length = len(Olenna_Words_String)
Olenna_String_Length

388

In [243]:
Olenna_Sentiment = TextBlob(Olenna_Words_String).sentiment
Olenna_Sentiment

Sentiment(polarity=0.196969696969697, subjectivity=0.5787878787878789)

### Samwell Tarly

In [244]:
Sam1 = season_seven.loc['Samwell']
Sam2 = season_seven.loc['Sam']

In [245]:
Sam_Text1 = Sam1.values.tolist()
Sam_Text2 = Sam2.values.tolist()

In [246]:
Sam_Words = []

for word in Sam_Text1:
    if len(word)>=1:
        if word != '0':
            Sam_Words.append(word)

for word in Sam_Text2:
    if len(word)>=1:
        if word != '0':
            Sam_Words.append(word)            
            
#Sam_Words  

In [247]:
Sam_Words_String = ' '.join(Sam_Words)
#Sam_Words_String

In [248]:
Sam_String_Length = len(Sam_Words_String)
Sam_String_Length

1190

In [249]:
Sam_Sentiment = TextBlob(Sam_Words_String).sentiment
Sam_Sentiment

Sentiment(polarity=-0.054761904761904776, subjectivity=0.5732142857142856)

### Sansa Stark

In [250]:
Sansa = season_seven.loc['Sansa']
#print Sansa

In [251]:
Sansa_Text = Sansa.values.tolist()
#Sansa_Text

In [252]:
Sansa_Words = []

for word in Sansa_Text:
    if len(word)>=1:
        if word != '0':
            Sansa_Words.append(word)
        
#Sansa_Words 

In [253]:
Sansa_Words_String = ' '.join(Sansa_Words)
#Sansa_Words_String

In [254]:
Sansa_String_Length = len(Sansa_Words_String)
Sansa_String_Length

2204

In [255]:
Sansa_Sentiment = TextBlob(Sansa_Words_String).sentiment
Sansa_Sentiment

Sentiment(polarity=-0.054665379665379675, subjectivity=0.6052445302445303)

### Theon Greyjoy

In [256]:
Theon = season_seven.loc['Theon']
#print Theon

In [257]:
Theon_Text = Theon.values.tolist()
#Theon_Text

In [258]:
Theon_Words = []

for word in Theon_Text:
    if len(word)>=1:
        if word != '0':
            Theon_Words.append(word)
        
#Theon_Words 

In [259]:
Theon_Words_String = ' '.join(Theon_Words)
#Theon_Words_String

In [260]:
Theon_String_Length = len(Theon_Words_String)
Theon_String_Length

71

In [261]:
Theon_Sentiment = TextBlob(Theon_Words_String).sentiment
Theon_Sentiment

Sentiment(polarity=0.35119047619047616, subjectivity=0.5178571428571428)

### Tyrion Lannister

In [262]:
Tyrion = season_seven.loc['Tyrion']
#print Tyrion

In [263]:
Tyrion_Text = Tyrion.values.tolist()
#Tyrion_Text

In [264]:
Tyrion_Words = []

for word in Tyrion_Text:
    if len(word)>=1:
        if word != '0':
            Tyrion_Words.append(word)
        
#Tyrion_Words 

In [265]:
Tyrion_Words_String = ' '.join(Tyrion_Words)
#Tyrion_Words_String

In [266]:
Tyrion_String_Length = len(Tyrion_Words_String)
Tyrion_String_Length

4013

In [267]:
Tyrion_Sentiment = TextBlob(Tyrion_Words_String).sentiment
Tyrion_Sentiment

Sentiment(polarity=0.11740798417269005, subjectivity=0.5029361161714102)

### Varys

In [268]:
Varys = season_seven.loc['Varys']
#print Varys

In [269]:
Varys_Text = Varys.values.tolist()
#Varys_Text

In [270]:
Varys_Words = []

for word in Varys_Text:
    if len(word)>=1:
        if word != '0':
            Varys_Words.append(word)
        
#Varys_Words 

In [271]:
Varys_Words_String = ' '.join(Varys_Words)
#Varys_Words_String

In [272]:
Varys_String_Length = len(Varys_Words_String)
Varys_String_Length

944

In [273]:
Varys_Sentiment = TextBlob(Varys_Words_String).sentiment
Varys_Sentiment

Sentiment(polarity=0.23273809523809524, subjectivity=0.5442460317460318)

### Yara Greyjoy

In [274]:
Yara = season_seven.loc['Yara']
#print Yara

In [275]:
Yara_Text = Yara.values.tolist()
#Yara_Text

In [276]:
Yara_Words = []

for word in Yara_Text:
    if len(word)>=1:
        if word != '0':
            Yara_Words.append(word)
        
#Yara_Words 

In [277]:
Yara_Words_String = ' '.join(Yara_Words)
#Yara_Words_String

In [278]:
Yara_String_Length = len(Yara_Words_String)
Yara_String_Length

125

In [279]:
Yara_Sentiment = TextBlob(Yara_Words_String).sentiment
Yara_Sentiment

Sentiment(polarity=0.0, subjectivity=0.0)